We start by cloning our example selection algorithm repository and installing some additional dependencies not preinstalled in Colab environments:

In [ ]:
!pip install -q fire

In [ ]:
!git clone https://github.com/harvard-edge/dataperf-speech-example/
import sys
sys.path.append("/content/dataperf-speech-example/")


#TODO

Download the dataperf data from an MLCommons bucket

In [ ]:
for lang in ["en", "id", "pt"]:
    !tar xf "dataperf_{lang}_data.tar.gz"

Now we will perform training set selection on the downloaded embeddings for each language, using our default selection algorithm (which simply performs crossfold-validation). The evaluation strategy can be changed by editing `dataperf-speech-example/workspace/dataperf_speech_config.yaml` 

The goal of this challenge is to add your own selection algorithm and outperform the provided baselines in both the minimal number of samples required and the final accuracy.

The selection algorithm will output a training file for each language, `en_train.json`, `id_train.json`, and `pt_train.json`.

These are the files you would upload to Dynabench for official evaluation, but in the next cell, we will run local unofficial evaluation using our provided evaluation data.

In [ ]:
for lang in ["en", "id", "pt"]:
    !cd dataperf-speech-example && \
        python -m selection.main \
        --language "{lang}" \
        --allowed_training_set "/content/dataperf_{lang}_data/allowed_training_set.yaml" \
        --train_embeddings_dir "/content/dataperf_{lang}_data/train_embeddings/" \
        --outdir "/content/"

Finally, let's run a local unofficial evaluation on the results of the training set selection algorithm (`en_train.json`, `id_train.json`, and `pt_train.json`). 

For each language, a balanced accuracy score will be printed out, using the evaluation data samples specified in `eval.yaml`:

```
validating selected IDs
loading selected training data
Loading targets: 100% 5/5 [00:00<00:00, 25.49it/s]
Loading nontargets: 100% 38/38 [00:00<00:00, 150.04it/s]
loading eval data
Loading targets: 100% 5/5 [00:00<00:00, 79.48it/s]
Loading nontargets: 100% 200/200 [00:14<00:00, 14.13it/s]
Score:  0.9174624016432674
```


In [ ]:
for lang in ["en", "id", "pt"]:
    !python dataperf-speech-example/eval.py \
      --language "{lang}" \
      --eval_embeddings_dir "/content/dataperf_{lang}_data/eval_embeddings/" \
      --train_embeddings_dir "/content/dataperf_{lang}_data/train_embeddings/" \
      --allowed_training_set "/content/dataperf_{lang}_data/allowed_training_set.yaml" \
      --eval_file "/content/dataperf_{lang}_data/eval.yaml" \
      --train_file "/content/{lang}_train.json" \
      --config_file dataperf-speech-example/workspace/dataperf_speech_config.yaml